
# 🚀 Colab Setup — **CNNs-distracted-driving** (hardcoded + config-aware)

This version is **simplified and hardcoded** for your repo and URL, and it **respects your `src/ddriver/config.py`**.
- Repo name fixed to **`CNNs-distracted-driving`**
- Repo URL fixed to **`https://github.com/ClaudiaCPach/CNNs-distracted-driving`**
- Uses your `config.py` convention: when running in Colab, we **set env vars** (`DRIVE_PATH`, `DATASET_ROOT`, `OUT_ROOT`, `CKPT_ROOT`, `FAST_DATA`) so your code reads correct paths via `ddriver.config`.
- Optional `FAST_DATA` at `/content/data` for faster I/O (if you later copy data there).

> Run cells **top → bottom** the first time. Re-run **Update repo** to pull new commits after you push.


In [ ]:

# 🔧 0) (Optional) quick GPU check
!nvidia-smi || echo "No GPU detected — CPU runtime is okay for setup steps."


In [ ]:

# 🔧 1) Fixed config for your repo + Drive layout
REPO_URL       = "https://github.com/ClaudiaCPach/CNNs-distracted-driving"
REPO_DIRNAME   = "CNNs-distracted-driving"   # hardcoded
BRANCH         = "main"
PROJECT_ROOT   = f"/content/{REPO_DIRNAME}"  # where the repo will live in Colab

# Your persistent Google Drive base folder (matches your project docs):
DRIVE_PATH     = "/content/drive/MyDrive/TFM"

# Your dataset lives under TFM/data/auc.distracted.driver.dataset_v2 (as per your structure)
DATASET_ROOT   = f"{DRIVE_PATH}/data/auc.distracted.driver.dataset_v2"
OUT_ROOT       = f"{DRIVE_PATH}/outputs"
CKPT_ROOT      = f"{DRIVE_PATH}/checkpoints"

# Optional: a fast, ephemeral workspace inside the VM
FAST_DATA      = "/content/data"   # leave as-is; you can rsync into this later for speed


In [ ]:

# 🔌 2) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
print("✅ Drive mounted.")


In [ ]:

# 📁 3) Clone or update the repo (no name inference — all hardcoded)
import os, subprocess

def sh(cmd):
    print(f"\n$ {cmd}")
    rc = subprocess.call(cmd, shell=True, executable="/bin/bash")
    if rc != 0:
        raise RuntimeError(f"Command failed with exit code {rc}: {cmd}")

if os.path.isdir(PROJECT_ROOT):
    print(f"📁 Repo already present at {PROJECT_ROOT}. Pulling latest on branch {BRANCH}...")
    sh(f"cd {PROJECT_ROOT} && git fetch origin {BRANCH} && git checkout {BRANCH} && git pull --rebase origin {BRANCH}")
else:
    print(f"⬇️ Cloning {REPO_URL} → {PROJECT_ROOT}")
    sh(f"git clone --branch {BRANCH} {REPO_URL} {PROJECT_ROOT}")

print("PROJECT_ROOT =", PROJECT_ROOT)


In [ ]:

# 📦 4) Install the repo (editable) + requirements (uses pyproject.toml if present)
import os, subprocess

def sh(cmd):
    print(f"\n$ {cmd}")
    rc = subprocess.call(cmd, shell=True, executable="/bin/bash")
    if rc != 0:
        raise RuntimeError(f"Command failed with exit code {rc}: {cmd}")

print("🔄 Upgrading pip/setuptools/wheel...")
sh("python -m pip install --upgrade pip setuptools wheel")

has_pyproject = os.path.exists(os.path.join(PROJECT_ROOT, "pyproject.toml"))
if has_pyproject:
    print("📦 Editable install from pyproject.toml ...")
    sh(f"cd {PROJECT_ROOT} && pip install -e .")
else:
    print("⚠️ No pyproject.toml found. Skipping editable install.")

req_path = os.path.join(PROJECT_ROOT, "requirements.txt")
if os.path.exists(req_path):
    print("📝 Installing requirements.txt...")
    sh(f"pip install -r {req_path}")
else:
    print("ℹ️ No requirements.txt found — continuing.")


In [ ]:

# 🌳 5) Configure environment for your ddriver.config (Colab branch)
# Your config.py reads env vars and falls back to sensible defaults when in Colab.
import os

os.environ["DRIVE_PATH"]   = DRIVE_PATH
os.environ["DATASET_ROOT"] = DATASET_ROOT
os.environ["OUT_ROOT"]     = OUT_ROOT
os.environ["CKPT_ROOT"]    = CKPT_ROOT
os.environ["FAST_DATA"]    = FAST_DATA

# Also write a .env (harmless in Colab; helpful if code calls load_dotenv())
env_text = f"""DRIVE_PATH={DRIVE_PATH}
DATASET_ROOT={DATASET_ROOT}
OUT_ROOT={OUT_ROOT}
CKPT_ROOT={CKPT_ROOT}
FAST_DATA={FAST_DATA}
"""
with open(os.path.join(PROJECT_ROOT, ".env"), "w") as f:
    f.write(env_text)

print("✅ Environment variables set for ddriver.config")
print("\nSummary:")
for k in ["DRIVE_PATH","DATASET_ROOT","OUT_ROOT","CKPT_ROOT","FAST_DATA"]:
    print(f"{k} = {os.environ[k]}")


In [ ]:

# 🔗 6) (Optional) Symlink dataset into repo for familiar paths (scripts that assume PROJECT_ROOT/data/...)
# Not required when using ddriver.config, but convenient for ad-hoc browsing.
import os

LOCAL_DATA_DIR = f"{PROJECT_ROOT}/data"
os.makedirs(LOCAL_DATA_DIR, exist_ok=True)

dataset_link = os.path.join(LOCAL_DATA_DIR, "auc.distracted.driver.dataset_v2")
if not os.path.islink(dataset_link) and not os.path.exists(dataset_link):
    try:
        os.symlink(DATASET_ROOT, dataset_link)
        print(f"🔗 Symlinked {dataset_link} → {DATASET_ROOT}")
    except OSError as e:
        print(f"ℹ️ Symlink skipped or failed: {e}")
else:
    print("ℹ️ Dataset link already exists.")


In [ ]:

# 🔍 7) Quick sanity checks
import os, glob

def preview_dir(path, n=10):
    print(f"Listing up to {n} items under: {path}")
    try:
        for i, name in enumerate(sorted(os.listdir(path))):
            print("  -", name)
            if i+1 >= n:
                break
    except Exception as e:
        print("Could not list:", e)

print("\nTop-level DATASET_ROOT:")
preview_dir(os.environ["DATASET_ROOT"], n=10)

cam1_train = os.path.join(os.environ["DATASET_ROOT"], "v2_cam1_cam2_split_by_driver", "Camera 1", "train")
print("\nCamera 1/train class folders (first 10):")
preview_dir(cam1_train, n=10)

for cls in ["c0","c1","c2"]:
    cls_dir = os.path.join(cam1_train, cls)
    if os.path.isdir(cls_dir):
        num_imgs = len([p for p in glob.glob(os.path.join(cls_dir, "*")) if os.path.isfile(p)])
        print(f"  • {cls}: {num_imgs} files")


In [ ]:

# ✅ 8) Import smoke test (uses your package + config.py)
import sys, os
sys.path.append(PROJECT_ROOT)

try:
    import ddriver
    print("ddriver imported OK from:", ddriver.__file__)
    # Confirm config picks up Colab env:
    try:
        from ddriver import config
        print("Loaded ddriver.config successfully.")
        # Echo the resolved paths from config (they are pathlib.Path objects)
        print("config.DATASET_ROOT =", config.DATASET_ROOT)
        print("config.OUT_ROOT     =", config.OUT_ROOT)
        print("config.CKPT_ROOT    =", config.CKPT_ROOT)
        print("config.FAST_DATA    =", config.FAST_DATA)
    except Exception as e:
        print("Note: ddriver.config not imported:", e)
except Exception as e:
    print("⚠️ Import failed — check package name/setup.")
    print(e)



### ✅ You’re set!
- Your repo + URL are **hardcoded**.
- `ddriver.config` will see the Colab env vars and resolve paths there.
- Re-run **Clone/Update** after pushing new commits.
- Optional: copy some data into `/content/data` to use `FAST_DATA` for speed, then call `ddriver.config.dataset_dir(prefer_fast=True)` in your scripts.
